In [2]:
from math import prod
from collections import defaultdict
from aocutils.common import ints
workflows, ratings = open("in.txt").read().split("\n\n")
flows = defaultdict(list)
for work in workflows.split('\n'):
    left, right = work.split('{')
    for rule in right[:-1].split(','):
        if ':' in rule:
            condition, outcome = rule.split(':')
            flows[left].append((condition[0], condition[1], ints(condition)[0], outcome))
        else:
            flows[left].append(rule)

122112157518711

In [11]:
state = ['in', {'x':(1,4000),'m':(1,4000),'a':(1,4000),'s':(1,4000)}]
tocheck = [state]
ans = 0
ops = 0
while tocheck:
    state = tocheck.pop()
    cur, xmas = state
    if cur == 'R':
        pass
    elif cur == 'A':
        ans += prod(map(lambda x: x[1]-x[0]+1, xmas.values()))
    else:
        for rule in flows[cur]:
            if isinstance(rule, tuple):
                letter, op, value, target = rule
                minn, maxx = xmas[letter]
                if op == '>':
                    xmas[letter] = [value + 1, maxx]
                    tocheck.append((target,xmas.copy()))
                    xmas[letter] = [minn, value]
                else:
                    xmas[letter] = [minn, value-1]
                    tocheck.append((target,xmas.copy()))
                    xmas[letter] = [value, maxx]
            else:
                tocheck.append((rule,xmas))
ans

122112157518711

In [27]:
# p1
import operator
ops = {'<':operator.lt, '>':operator.gt}

ans = 0
for part in ratings.split('\n'):
    x,m,a,s = ints(part)
    var = {'m':m, 'x':x, 'a':a, 's':s}
    
    cur = 'in'
    while True:
        currule = flows[cur]
        for rule in currule:
            if isinstance(rule, tuple):
                if ops[rule[1]](var[rule[0]], rule[2]):
                    cur = rule[3]
                    break
            else:
                cur = rule
        if cur == 'R':
            break
        if cur == 'A':
            ans += sum(ints(part))
            break
ans

374873

In [28]:
# below is my original code. essentially I misunderstood below snippet:
variable = 1
d = {'key': variable}
variable = 2
d['key'] == 1 # not 2


True

In [29]:
# o
state = ['in', (1,4000),(1,4000),(1,4000),(1,4000)]
tocheck = [state]
var2idx = {'x':1, 'm':2, 'a':3, 's':4}
ans = 0
nans = 0
found = []
from copy import deepcopy
while tocheck:
    state = tocheck.pop()
    cur, x,m,a,s = state
    var = {'m':m, 'x':x, 'a':a, 's':s}
    currule = flows[cur]
    if cur == 'R':
        options = 1
        for var in [x,m,a,s]:
            options *= (var[1]-var[0]+1)
        nans += options
        continue
    elif cur == 'A':
        found.append(state)
        options = 1
        for var in [x,m,a,s]:
            options *= (var[1]-var[0]+1)
        ans += options
        continue
    
    for rule in currule:
        cur, x,m,a,s = state
        assert cur != 'A' and cur != 'R'
        if isinstance(rule, tuple):
            minn, maxx = var[rule[0]] # incorrect, took me 1,5 hours of debugging
            minn, maxx = state[var2idx[rule[0]]] # correct
            assert minn < maxx
            # minn, maxx = var[rule[0]]
            assert minn < rule[2] < maxx
            if rule[1] == '>':
                if rule[2] > maxx:
                    break
                elif rule[2] <= minn:
                    state[0] = rule[3]
                    tocheck.append(state)
                    continue
                else:
                    assert minn < rule[2] < maxx
                    state[var2idx[rule[0]]] = (rule[2] + 1, maxx)
                    state[0] = rule[3]
                    tocheck.append(deepcopy(state))
                    state[0] = cur
                    state[var2idx[rule[0]]] = (minn, rule[2])
            else:
                assert rule[1] == '<'
                if rule[2] < minn:
                    print('hi')
                    break
                elif rule[2] >= maxx:
                    print('hi')
                    state[0] = rule[3]
                    tocheck.append(state)
                    continue
                else:
                    assert minn < rule[2] < maxx
                    state[var2idx[rule[0]]] = (minn, rule[2]-1)
                    state[0] = rule[3]
                    tocheck.append(deepcopy(state))
                    state[0] = cur
                    state[var2idx[rule[0]]] = (rule[2], maxx)
        else:
            state[0] = rule
            tocheck.append(deepcopy(state))
            break

ans, ans + nans == 4000**4
    

(122112157518711, True)

1